# Inhibition effects in ProbLog

An important goal of statistical relational learning formalisms is to develop representations that are compact and expressive but also easy to read and maintain. This is can be achieved by exploiting the modularity of rule-based structures and is related to the noisy-or structure where parents independently influence a joint effect. Typically, these rules are combined in an additive manner where a new rule increases the probability of the effect.

In this example, we show a language feature for ProbLog, where we allow negation in the head of rules to express the inhibition of an effect in a modular manner. This is a generalization of the inhibited noisy-or structure that can deal with cycles and, foremost, is non-conflicting. It is a natural counterpart to the standard noisy-or.

Such a rule looks like:

    0.7::a :- cause_a.
    0.4::\+a :- inhibit_a.

Based on:

- Meert, Wannes, and Joost Vennekens. "Inhibited effects in CP-logic." Probabilistic Graphical Models. Springer International Publishing, 2014. 350-365.


In [1]:
import sys, os
sys.path.append(os.path.abspath('../../util'))
%load_ext problog.magic

## Example 1

Suppose we want to express two independent causes `c1` and `c2` that make an outcome `e` more likely. This is the typical case for a **noisy-or** gate:

In [2]:
%%problog
0.5::c1. 0.5::c2.

0.3::e :- c1.
0.4::e :- c2.
            
evidence(c1,true).
evidence(c2,true).
            
query(e).

Atom Probability e 0.5800

In some cases, however, if both causes are present this reduces the likelihood of the effect happening. For example when the presence of a second drug blocks the impact of the original drug. In such a case we can use negated heads. Depending on the chosen probabilities the effect can be lower than the joint occurance of `c1` and `c2` or be lower then either `p1` or `p2`:

In [3]:
%%problog
0.5::c1. 0.5::c2.

0.3::e1 :- c1.
0.2::\+e1 :- c2.

0.4::e2 :- c2.
0.2::\+e2 :- c1.

e :- e1.
e :- e2.

evidence(c1,true).
evidence(c2,true).
         
query(e).

Atom Probability e 0.4832

If, like in the example above, the effect is only present when both causes are present, we can simplify the theory to:

In [4]:
%%problog
0.5::c1. 0.5::c2.

0.3::e :- c1.
0.4::e :- c2.
            
0.166666::\+e :- c1, c2.

evidence(c1,true).
evidence(c2,true).
         
query(e).

Atom Probability e 0.4833

Note that this solution reduces the number of parameters but is less modular if we would want to extend the example to three causes (you need to change the rule with `c1,c2` instead of simply adding new rules).

## Example 2

An infectious disease spreads through a population as follows: when- ever two people are in regular contact with each other and one is infected, there is a probability of 0.6 of the infection spreading also to the other person. Given a set of initially infected people and a graph of connections between individuals in the population, the goal is to predict the spread of the disease.

In [5]:
%%problog
person(a).
person(b).

0.1::initialInf(X) :- person(X).
0.1::inf(X) :- person(X).
0.1::contact(X,Y) :- person(X), person(Y).

inf(X)      :- initialInf(X).
0.6::inf(X) :- contact(X, Y), inf(Y).
            
query(inf(_)).

Atom Probability inf(a) 0.1992 inf(b) 0.1992

Given any set of individuals and any interpretation for the exogenous predicates InitialInf and Contact, this ProbLog program defines the probability with which each individual will be infected. In particular, no restrictions (such as acyclicity) are imposed on the Contact-relation.

In addition to representing probability distributions in a compact way, ProbLog also aims at being elaboration tolerant: once a program for a given domain has been constructed, it should be easy to adapt this theory when we learn new facts about the domain. Ideally, new knowledge should be incorporated in a way which respects the inherent modularity of CP-logic, in the sense that it may involve adding or removing rules, but not changing existing rules.

One such operation for which ProbLog is obviously well-suited is when a new cause for some effect is discovered. For instance, suppose we learn that, in addition to being among the initially infected and having contact with in- fected individuals from the population, people may also contract the disease by travelling to particular locations (e.g., with probability 0.2). We can update our program accordingly, by simply adding an additional rule:

In [6]:
%%problog
person(a).
person(b).

0.1::initialInf(X) :- person(X).
0.1::inf(X) :- person(X).
0.1::contact(X,Y) :- person(X), person(Y).
0.1::riskyTravel(X) :- person(X).

inf(X)      :- initialInf(X).
0.6::inf(X) :- contact(X, Y), inf(Y).
0.2::inf(X) :- riskyTravel(X).
            
query(inf(_)).

Atom Probability inf(a) 0.2160 inf(b) 0.2160

Importantly, there is no need to change our existing rules.

A second operation is discovering that certain parts of the population form an exception to the general rules. For instance, suppose that certain people are discovered to be especially susceptible (e.g., probability 0.8) to contracting the disease through contact with an already infected person. We can represent this by “case splitting” rule (2) into the following two rules:

In [7]:
%%problog
person(a).
person(b).

0.1::initialInf(X) :- person(X).
0.1::inf(X) :- person(X).
0.1::contact(X,Y) :- person(X), person(Y).
0.1::riskyTravel(X) :- person(X).
0.1::susceptible(X) :- person(X).

inf(X)      :- initialInf(X).
0.6::inf(X) :- contact(X, Y), inf(Y), \+susceptible(X).
0.8::inf(X) :- contact(X, Y), inf(Y),   susceptible(X).
0.2::inf(X) :- riskyTravel(X).
            
query(inf(_)).

Atom Probability inf(a) 0.2163 inf(b) 0.2163

However, this solution has the downside that it forces us to change an existing rule. A better alternative is to exploit the additive nature of different causes for the same effect in ProbLog:

In [8]:
%%problog
person(a).
person(b).

0.1::initialInf(X) :- person(X).
0.1::inf(X) :- person(X).
0.1::contact(X,Y) :- person(X), person(Y).
0.1::riskyTravel(X) :- person(X).
0.1::susceptible(X) :- person(X).

inf(X)      :- initialInf(X).
0.6::inf(X) :- contact(X, Y), inf(Y).
0.5::inf(X) :- contact(X, Y), inf(Y),   susceptible(X).
0.2::inf(X) :- riskyTravel(X).
            
query(inf(_)).

Atom Probability inf(a) 0.2163 inf(b) 0.2163

For non-susceptible individuals, only the first rule is applicable, so they still get infected with the same probability of 0.6 as before. The same rule of course also applies to susceptible individuals, whom the second rule then gives an *additional* probability of getting infected *because* they are susceptible. This brings their total probability of being infected up to 0.6 + (1 − 0.6) · 0.5 = 0.8. When compared to the “case splitting” theory, this representation has the advantage that it allows the “default” rule for normal people to remain unchanged.

In addition to discovering that certain parts of the population are especially susceptible to the infection, it is equally possible to discover that certain people tend to be more resistant to it. Again, this can be solved by case splitting:

In [9]:
%%problog
person(a).
person(b).

0.1::initialInf(X) :- person(X).
0.1::inf(X) :- person(X).
0.1::contact(X,Y) :- person(X), person(Y).
0.1::riskyTravel(X) :- person(X).
0.1::resistant(X) :- person(X).

inf(X)      :- initialInf(X).
0.6::inf(X) :- contact(X, Y), inf(Y), \+resistant(X).
0.4::inf(X) :- contact(X, Y), inf(Y),   resistant(X).
0.2::inf(X) :- riskyTravel(X).
            
query(inf(_)).

Atom Probability inf(a) 0.2157 inf(b) 0.2157

A solution in which we can keep our original “default” rule unchanged is not possible using noisy-or or is not intuitive to impossible in current probabilistic logics. Indeed, this is an obvious consequence of the fact that adding additional rules can only increase probabilities. In this paper, we introduce the new feature of negation in the head of rules, which will allow us to represent also a decrease in probabilities. In particular, we will be able to represent our example as:

In [10]:
%%problog
person(a).
person(b).

0.1::initialInf(X) :- person(X).
0.1::inf(X) :- person(X).
0.1::contact(X,Y) :- person(X), person(Y).
0.1::riskyTravel(X) :- person(X).
0.1::resistant(X) :- person(X).

inf(X)         :- initialInf(X).
0.6::inf(X)    :- contact(X, Y), inf(Y).
0.33::\+inf(X) :- resistant(X).
0.2::inf(X)    :- riskyTravel(X).
            
query(inf(_)).

Atom Probability inf(a) 0.2086 inf(b) 0.2086

## More information

https://dtai.cs.kuleuven.be/problog

(c) 2015, DTAI, KU Leuven